In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
from pymystem3 import Mystem
import re

In [2]:
habr_clusters = pd.read_csv('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/clustering/habs_clusters.csv', index_col=0)
habr_info = pd.read_json('/Users/alexandrasorokovikova/Data Science/НИР Сравнение IT/Comparison-of-IT-trends/data_parsing/data/habr.json')


In [3]:
habr_info.head()

,time,tags,habs,saved,comments,views
0,2016-04-11T09:29:03.000Z,"[безопасная динамичность, brave, блокировка ре...",[управление медиа],4,0,1749
1,2016-04-09T14:19:17.000Z,"[dash, блокчейн, криптовалюта, управление]","[фриланс, венчурные инвестиции]",12,7,10370
2,2016-04-10T05:11:42.000Z,"[mail.ru group, headhunter, российский рынок, ...","[карьера в it-индустрии, бизнес-модели]",12,1,5472
3,2016-04-11T17:05:54.000Z,"[tidal, jay-z, канье уэст, стриминговые сервисы]",[монетизация веб-сервисов],3,1,3706
4,2016-04-09T14:52:55.000Z,"[mozilla, brave, блокировщики рекламы, судебно...",[законодательство в it],4,7,6244


In [4]:
habr_clusters.head()

,hab,agglomerative_cluster
0,компания,1
1,блог,1
2,разработка,7
3,программирование,4
4,веб,7


## Выделение категорий
Для выделения категории из кластеров выберем наиболее встречающийся hab

In [5]:
stem = Mystem()

def count_categories(items, data):
    habs_counter = Counter()
    
    for row in tqdm(data.index):
        
        habs_data = data[row]
        habs_data = ' '.join(stem.lemmatize(' '.join(habs_data)))
        habs_data = re.sub("[^A-Za-zА-Яа-я\s]", '', habs_data)
        habs_data = habs_data.split()
        
        for item in items:
            if len(item.split('_')) == 2:
                item_splited = item.split('_')
                if item_splited[0] in habs_data and item_splited[1] in habs_data:
                    habs_counter[item] += 1
            else:
                if item in habs_data:
                    habs_counter[item] += 1
                    
    return habs_counter
        
habs_counter = count_categories(habr_clusters['hab'], habr_info['habs'])

100%|██████████| 38071/38071 [00:37<00:00, 1002.69it/s]


In [6]:
habs_counter

Counter({'управление': 3223,
         'медиа': 145,
         'венчурный_инвестиция': 157,
         'фриланс': 146,
         'it': 9074,
         'карьера': 1621,
         'индустрия': 1617,
         'бизнес_модель': 469,
         'бизнес': 470,
         'веб': 3028,
         'сервис': 1292,
         'монетизация': 300,
         'законодательство': 1527,
         'финансы': 945,
         'патентование': 64,
         'интернет': 860,
         'маркетинг': 558,
         'контекстный_реклама': 108,
         'gtd': 211,
         'исследование_прогноз': 750,
         'исследование': 750,
         'повышение_конверсия': 143,
         'контент': 196,
         'growth_hacking': 80,
         'проект': 1181,
         'продукт': 814,
         'конференция': 887,
         'система': 2712,
         'игра': 1935,
         'e_commerce': 420,
         'e': 420,
         'читальный_зал': 904,
         'продажа': 232,
         'аналитика': 159,
         'учебный_процесс': 888,
         'процесс': 888,
  

In [7]:
def count_habs(row):
    return habs_counter[row]

habr_clusters['frequency'] = habr_clusters['hab'].apply(count_habs)
habr_clusters

,hab,agglomerative_cluster,frequency
0,компания,1,2328
1,блог,1,497
2,разработка,7,8178
3,программирование,4,5184
4,веб,7,3028
...,...,...,...
1704,1c,1,0
1705,smmframe,0,0
1706,связной,0,0
1707,экспресс,0,0


In [17]:
habr_clusters[habr_clusters['agglomerative_cluster'] == 8].sort_values(by = 'frequency', ascending = False).head(10)

,hab,agglomerative_cluster,frequency
49,машинный_обучение,8,1609
24,data,8,972
48,big,8,692
57,mining,8,394
101,визуализация_данные,8,245
180,открытый_данные,8,103
170,r,8,100
386,science,8,3
337,lab,8,0
355,new_professions,8,0
